author：王尧
测试部署的组件是否在正常运行

In [37]:
# 打印pyspark的版本
import pyspark
pyspark.__version__

'2.4.5'

In [1]:
# 尝试交互式操作，注意这里和后面的步骤都是在远方服务器执行，所以不需要再配置路径
# spark在yarn下，默认会去找服务器的SPARK_HOME环境变量
from pyspark import SparkContext
sc = SparkContext("yarn", "test")
sc

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/29 13:54:19 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


<SparkContext master=yarn appName=test>

In [2]:
# 测试数字
lines = sc.parallelize(range(100))
",".join([str(i) for i in lines.collect()])

'0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99'

In [4]:
lines_2 = lines.map(lambda x : x*2)
",".join([str(i) for i in lines_2.collect()])

'0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74,76,78,80,82,84,86,88,90,92,94,96,98,100,102,104,106,108,110,112,114,116,118,120,122,124,126,128,130,132,134,136,138,140,142,144,146,148,150,152,154,156,158,160,162,164,166,168,170,172,174,176,178,180,182,184,186,188,190,192,194,196,198'

In [64]:
# 显示hdfs上的文件
import pyhdfs
fs = pyhdfs.HdfsClient(user_name='zzti')
print(fs.listdir("/"))
print(list(fs.walk("/test")))
print(fs.listdir("/test/input"))
for line in fs.open('/test/input/i_have_a_dream.data'):
    print(line)
fs.delete("/test/output",recursive=True)
print(list(fs.walk("/test")))

['data', 'hbase', 'test', 'tmp', 'user']
[('/test', ['input', 'output'], []), ('/test/input', [], ['i_have_a_dream.data']), ('/test/output', [], ['_SUCCESS', 'part-00000', 'part-00001'])]
['i_have_a_dream.data']
b'I am happy to join with you today in what will go down in history as the greatest demonstration for freedom in the history of our nation.\n'
b'Five score years ago, a great American, in whose symbolic shadow we stand today, signed the Emancipation Proclamation. This momentous decree came as a great beacon light of hope to millions of Negro slaves who had been seared in the flames of withering injustice. It came as a joyous daybreak to end the long night of their captivity.\n'
b"But one hundred years later, the Negro still is not free. One hundred years later, the life of the Negro is still sadly crippled by the manacles of segregation and the chains of discrimination. One hundred years later, the Negro lives on a lonely island of poverty in the midst of a vast ocean of materi

In [68]:
# 测试wordcount
rdd = sc.textFile("/test/input/i_have_a_dream.data")
result1 = rdd.flatMap(lambda x: x.split(" "))\
    .map(lambda x: (x, 1))\
    .groupByKey()\
    .map(lambda x: (x[0], len(x[1])))
result1.collect()

[('am', 2),
 ('join', 2),
 ('in', 19),
 ('go', 8),
 ('down', 4),
 ('as', 20),
 ('greatest', 1),
 ('of', 97),
 ('score', 1),
 ('years', 5),
 ('ago,', 1),
 ('whose', 1),
 ('shadow', 1),
 ('we', 14),
 ('stand', 3),
 ('today,', 3),
 ('Emancipation', 1),
 ('Proclamation.', 1),
 ('decree', 1),
 ('came', 2),
 ('millions', 1),
 ('slaves', 2),
 ('flames', 1),
 ('withering', 1),
 ('It', 4),
 ('daybreak', 1),
 ('end', 1),
 ('long', 6),
 ('night', 1),
 ('But', 4),
 ('is', 23),
 ('One', 3),
 ('crippled', 1),
 ('segregation', 2),
 ('lonely', 1),
 ('midst', 1),
 ('ocean', 1),
 ('material', 1),
 ('prosperity.', 1),
 ('corners', 1),
 ('American', 3),
 ('society', 1),
 ('finds', 1),
 ('an', 4),
 ('his', 3),
 ('own', 1),
 ('And', 11),
 ('shameful', 1),
 ('condition.', 1),
 ('cash', 2),
 ('check.', 1),
 ('When', 1),
 ('architects', 1),
 ('Independence,', 1),
 ('signing', 1),
 ('promissory', 2),
 ('was', 2),
 ('fall', 1),
 ('heir.', 1),
 ('men,', 2),
 ('yes,', 1),
 ('would', 2),
 ('guaranteed', 1),
 ('"una

In [74]:
# 测试写入HDFS
dir_output="/test/output/"
if fs.exists(dir_output):
    fs.delete(dir_output,recursive=True)
result1.saveAsTextFile(dir_output)
for file in fs.listdir(dir_output):
    for line in fs.open(dir_output+file,):
        print(str(line, encoding="utf-8"))

('Let', 9)

('us', 4)

('wallow', 1)

('in', 19)

('valley', 3)

('of', 97)

('say', 2)

('today,', 3)

('And', 11)

('even', 2)

('though', 1)

('we', 14)

('difficulties', 1)

('have', 17)

('dream.', 2)

('It', 4)

('is', 23)

('dream', 9)

('American', 3)

('this', 14)

('nation', 7)

('live', 2)

('out', 3)

('true', 1)

('meaning', 1)

('these', 1)

('truths', 1)

('are', 7)

('created', 1)

('slaves', 2)

('owners', 1)

('able', 6)

('down', 4)

('at', 1)

('brotherhood.', 3)

('state', 2)

('oppression,', 1)

('transformed', 1)

('into', 4)

('an', 4)

('four', 1)

('where', 3)

('color', 2)

('skin', 1)

('but', 2)

('today!', 2)

('Alabama,', 2)

('vicious', 1)

('racists,', 1)

('governor', 1)

('his', 3)

('lips', 1)

('"nullification"', 1)

('right', 1)

('there', 5)

('boys', 2)

('girls', 2)

('join', 2)

('as', 20)

('brothers.', 1)

('exalted,', 1)

('low,', 1)

('crooked', 1)

('straight;', 1)

('"and', 1)

('glory', 1)

('Lord', 1)

('revealed', 1)

('together."2', 1

In [5]:
# 终止测试
sc.stop()



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/25 13:30:13 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


Py4JError: org.apache.spark.api.python.PythonUtils.isEncryptionEnabled does not exist in the JVM